## Part IV:  Features and Labeling Functions

## Generating Features

In [1]:
# # If necessary:
# import os
# os.remove('snorkel.db');
# os.system('cp snorkel.db\ labels snorkel.db');

In [2]:
# %load_ext autoreload
# %autoreload 2

# from snorkel import SnorkelSession
# session = SnorkelSession()

In [3]:
# from snorkel.models import candidate_subclass

# Part_Temp = candidate_subclass('Part_Temp', ['part','temp'])

### Loading the `CandidateSet`

In [4]:
# from snorkel.models import CandidateSet
# train = session.query(CandidateSet).filter(
#     CandidateSet.name == 'Hardware Training Candidates').one()
# dev = session.query(CandidateSet).filter(
#     CandidateSet.name == 'Hardware Development Candidates').one()

### Creating feature matrix

In [5]:
# from snorkel.annotations import FeatureManager

# feature_manager = FeatureManager()

In [6]:
# %time F_train = feature_manager.create(session, train, 'Train Features')

In [7]:
# %time F_dev = feature_manager.update(session, dev, 'Train Features', expand_key_set=False)

### Reloading feature matrix

In [8]:
# %time F_train = feature_manager.load(session, train, 'Train Features')

In [9]:
# F_train

In [10]:
# F_train.get_candidate(0)

In [11]:
# F_train.get_key(0)

In [12]:
# from snorkel.utils import get_keys_by_candidate

# for f in get_keys_by_candidate(F_train, F_train.get_candidate(1))[:10]: print f

### Saving feature matrices

In [13]:
# # If necessary:
# import os
# os.system('cp snorkel.db snorkel.db\ featurized');

## Applying LFs

In [14]:
# If necessary:
import os
os.remove('snorkel.db');
os.system('cp snorkel.db\ featurized snorkel.db');

from snorkel import SnorkelSession
session = SnorkelSession()

from snorkel.models import candidate_subclass
Part_Temp = candidate_subclass('Part_Temp', ['part','temp'])

from snorkel.models import CandidateSet
train = session.query(CandidateSet).filter(
    CandidateSet.name == 'Hardware Training Candidates').one()

In [15]:
from snorkel.annotations import LabelManager

label_manager = LabelManager()

### Defining Labeling Functions

In [16]:
from snorkel.lf_helpers import *
from itertools import chain

LFs = []

# gold_file = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware_gold.csv'
# def LF_training_gold(c):
#     training_gold_dict = get_gold_dict(gold_file, 'stg_temp_min')
#     return 1 if ((c[0].parent.document.name).upper(), 
#                  (c[0].get_span()).upper(), 
#                  (''.join(c[1].get_span().split())).upper()) in training_gold_dict else 0
# LFs.append(LF_training_gold)

# from hardware_utils import load_extended_parts_dict
# gold_file ='data/hardware/hardware_gold.csv'
# parts_dict = load_extended_parts_dict(gold_file)
# print "Loaded %d part numbers." % len(parts_dict)

# def LF_parts_dict(c):
#     return 1 if (c.temp.parent.words)[0] in parts_dict else -1
# LFs.append(LF_foo)

# POSITIVE

def LF_to_right(c):
    return 1 if 'to' in get_right_ngrams(c.temp, window=1, n_max=1) else 0
LFs.append(LF_to_right)

def LF_150_right(c):
    return 1 if '150' in get_right_ngrams(c.temp, window=4, n_max=1) else 0
LFs.append(LF_150_right)

def LF_storage_row(c):
    return 1 if 'storage' in get_row_ngrams(c.temp, n_max=2) else 0
LFs.append(LF_storage_row)

def LF_operating_row(c):
    return 1 if 'operating' in get_row_ngrams(c.temp, n_max=2) else 0
LFs.append(LF_operating_row)

def LF_C_phrase(c):
    ngrams =  get_phrase_ngrams(c.temp, n_min=2, n_max=2, case_sensitive=True)
    return 1 if ('% C' in ngrams or '%C' in ngrams) else 0
LFs.append(LF_C_phrase)

def LF_temperature_row(c):
    return 1 if 'temperature' in get_row_ngrams(c.temp) else 0
LFs.append(LF_temperature_row)

def LF_tstg_row(c):
    row_ngrams = get_row_ngrams(c.temp, n_max=1)
    return 1 if ('tstg' in row_ngrams or 
                 'ts' in row_ngrams or 
                 't stg' in row_ngrams or
                 'stg' in row_ngrams) else 0
LFs.append(LF_tstg_row)


# NEGATIVE

def LF_temp_outside_table(c):
    return -1 if c.temp.parent.row is None else 0
LFs.append(LF_temp_outside_table)

def LF_collector_row(c):
    ngrams = get_row_ngrams(c.temp, n_max=1)
    return -1 if ('collector' in ngrams or
                  'v ceo' in ngrams or
                  'collector-emitter' in ngrams) else 0
LFs.append(LF_collector_row)

def LF_typ_cell(c):
    cell_ngrams = get_cell_ngrams(c.temp, n_max=1)
    row_ngrams = get_row_ngrams(c.temp, n_max=1)
    return -1 if ('typ' in cell_ngrams or
                  'typ.' in cell_ngrams or
                  'typ' in row_ngrams or
                  'typ.' in row_ngrams) else 0
LFs.append(LF_typ_cell)

def LF_test_phrase(c):
    return -1 if 'test' in get_phrase_ngrams(c.temp, n_max=1) else 0
LFs.append(LF_test_phrase)

def LF_condition_phrase(c):
    return -1 if 'condition' in get_phrase_ngrams(c.temp, n_max=1) else 0
LFs.append(LF_condition_phrase)

def LF_complement_left_row(c):
    return -1 if 'complement' in chain.from_iterable(
        [get_left_ngrams(c.part, window=10, n_max=1), get_row_ngrams(c.part, n_max=1)]) else 0
LFs.append(LF_complement_left_row)

def LF_voltage_row_temp(c):
    return -1 if ('voltage' in get_row_ngrams(c.temp, n_max=1, infer=True)) else 0
LFs.append(LF_voltage_row_temp)

def LF_voltage_row_part(c):
    return -1 if ('voltage' in get_row_ngrams(c.part, n_max=1)) else 0
LFs.append(LF_voltage_row_part)

def LF_v_row(c):
    return -1 if 'v' in get_row_ngrams(c.temp, infer=True) else 0
LFs.append(LF_v_row)

# def LF_min_col(c):
#     return -1 if 'min' in get_col_ngrams(c.temp) or 'min.' in get_col_ngrams(c.temp) else 0
# LFs.append(LF_min_col)

def LF_vceo_cell(c):
    return -1 if 'vceo' in get_cell_ngrams(c.temp) else 0
LFs.append(LF_vceo_cell)

def LF_too_many_numbers_row(c):
    num_numbers = list(get_row_ngrams(c.temp, attrib="ner_tags")).count('number')
    return -1 if num_numbers >= 3 else 0
LFs.append(LF_too_many_numbers_row)

### Testing LFs and LF_helpers

In [17]:
from snorkel.lf_helpers import get_right_ngrams, get_left_ngrams
from snorkel.lf_helpers import contains_token, contains_regex
from snorkel.lf_helpers import get_phrase_ngrams, get_cell_ngrams, get_neighbor_cell_ngrams
from snorkel.lf_helpers import get_row_ngrams, get_col_ngrams, get_aligned_ngrams
from snorkel.lf_helpers import same_document, same_table, same_cell, same_phrase
c = train[12]
print c.part
print c.temp
print c.temp.parent
print c.temp.parent.table
print same_document(c)
print same_table(c)
print contains_regex(c, rgx=r'849')


Span("BC182", parent=84426, chars=[0,4], words=[0,0])
ImplicitSpan("-55", parent=84276, words=[0,0], position=[0])
Phrase(Doc: BC182-D, Table: 0, Row: 7, Col: 2, Position: 0, Text: -55 to +150)
Table(Doc: BC182-D, Position: 0)
True
False
False


In [18]:
%time L_train = label_manager.create(session, train, 'LF Labels', f=LFs)
L_train

[========================================] 100%

Loading sparse Label matrix...
CPU times: user 13min 20s, sys: 1min 37s, total: 14min 57s
Wall time: 14min 34s


<75908x18 sparse matrix of type '<type 'numpy.float64'>'
	with 129814 stored elements in Compressed Sparse Row format>

In [19]:
train_gold = session.query(CandidateSet).filter(
    CandidateSet.name == 'Hardware Training Candidates -- Silver').one()

In [20]:
# L_train.lf_stats()
%time L_train.lf_stats(train_gold)

[========================================] 100%

CPU times: user 8.64 s, sys: 1.33 s, total: 9.98 s
Wall time: 8.48 s


,accuracy,conflicts,coverage,j,overlaps
LF_to_right,1.000000,0.014056,0.101649,0,0.101649
LF_150_right,1.000000,0.011448,0.107551,1,0.107551
LF_storage_row,1.000000,0.023199,0.258906,2,0.258906
LF_operating_row,1.000000,0.007931,0.064736,3,0.064736
LF_C_phrase,0.312888,0.237221,0.260961,4,0.260961
LF_temperature_row,1.000000,0.027823,0.301431,5,0.268997
LF_tstg_row,1.000000,0.001133,0.024134,6,0.024134
LF_temp_outside_table,0.756058,0.237248,0.369658,7,0.239882
LF_collector_row,NaN,0.000000,0.000000,8,0.000000
LF_typ_cell,NaN,0.000000,0.000000,9,0.000000


In [21]:
# If necessary
import os
os.system('cp snorkel.db snorkel.db\ features');

Next, in Part 5, we will test our model on the development `CandidateSet`.

# TEMPORARY

In [22]:
# print c.temp
# print c.temp.parent

In [23]:
# print list(get_row_ngrams(c.temp, attrib="ner_tags"))
# print list(get_row_ngrams(c.temp, attrib="words"))

In [24]:
# from pprint import pprint
# print(list(get_row_ngrams(c.temp, attrib="ner_tags")).count('o'))

In [25]:
# print LF_v_row(c)